In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch 
from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

from scipy.stats import spearmanr


import os
from collections import Counter

1. Load the dataset 

In [2]:
#load training data (will be put in a function later)  
path = os.getcwd()
for i in range(3) :

    path = os.path.dirname(path)

path += '/data/'
train_df = pd.read_csv(path + 'train_v1.csv',index_col="seq_id")
train_df = train_df.drop(columns=['data_source'])
train_df = train_df.dropna()
train_df.head()

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


2. Translate Amino-acids to numbers and create a One-Channel array for each sequence

In [3]:
#add a new column that contains the length of each protein sequence (before padding)
train_df['length'] = train_df['protein_sequence'].str.len()

In [4]:
# Function to encode sequences (one hot encoding)
max_length = max(train_df['length'])
def encode_seq(sequence):
    alphabet = ['A', 'C', 'D', 'E', 'F', 'G','H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] # aa letters
    char_to_int = dict((c, i) for i, c in enumerate(alphabet)) 
    integer_encoded = [char_to_int[char] for char in sequence] #each character becomes int
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(alphabet))] #0 for all letters
        letter[value] = 1 #modify the column corresponding to the letter to 1
        onehot_encoded.append(letter) #put in the array (1 letter = 1 array of 20 columns)
    
    ar =   np.transpose(np.array(onehot_encoded))
    
    zeros = np.zeros([len(alphabet),max_length   - len(integer_encoded)] )

    onehot_encoded = np.concatenate((ar, zeros), axis = 1) #zero padding

    

    return onehot_encoded #we have all arrays, corresponding to the whole sequence


# new column with encoded sequence (apply for each sequence)
train_df['encoded_sequence'] = train_df['protein_sequence'].apply(lambda x: encode_seq(x))

In [5]:
train_df['encoded_sequence'].values.shape

(28695,)

In [6]:
#final dataframe 
train_df.iloc[0]['encoded_sequence'].shape

(20, 8798)

In [7]:
max_length = max(train_df['length'])
max_length

8798

# Adding inverted sequences (DISCRADED) 
**Add it to report as an experiment ?**


inverted_df = train_df.copy()

#inver the protein sequence
inverted_df['protein_sequence'] = inverted_df.apply(lambda row : row['protein_sequence'][::-1],axis=1)
inverted_df['numerical_sequence'] = inverted_df.apply(lambda row : chr_to_int(row['protein_sequence']),axis=1)

# merge the original and inverted dataframes (size of dataset is doubled)
train_df = pd.concat([train_df,inverted_df],ignore_index=True)

## 3. Padding with zeros 

## add 0 to numerical_sequence to make all of them the same length
padded_train_df = train_df.copy()
padded_train_df['encoded_sequence'] = train_df.apply(lambda row : row['encoded_sequence'][:] + [0]*(max_length - row['length']),axis=1)

#Prepare dataframe to Pass to the dataloader (will be put in a function later)
padded_train_df.drop(columns=['protein_sequence'],inplace=True)
padded_train_df['y'] = padded_train_df['tm']
padded_train_df.drop(columns=['tm'],inplace=True)
padded_train_df['numerical_sequence'] = padded_train_df.apply(lambda row : np.array(row['numerical_sequence']),axis=1)

In [8]:
df = train_df.copy()

Split to train and validation sets

In [9]:
#splot padded_train_df into train and validation sets (will be put in a function later)
train_df = df.sample(frac=0.8,random_state=200)
val_df = df.drop(train_df.index)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


,protein_sequence,pH,tm,length,encoded_sequence
0,MPQESSDAKRIEPPRKVDFMLKPRFTNTVPDVPFDAKFMTCPFVPL...,7.0,34.3,424,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
1,MIKNIVISFEEQKEESRGSVEFQVFSFTNKIRRLTSHLELHRKDYL...,7.0,35.5,87,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,MASYPGPGKSKAKYPFKKRAGLQASAAAPEARSGLGASPLQSARSL...,7.0,55.6,823,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,MSRGSRFVRAPGLLLCRVNLQPQPKIPSFSYSLRSDYRLHNGFSNY...,7.0,43.3,727,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,MFWAKEEEENSLMRLLKTDNFTLEDVLLNEFVVQESRYGKAELVQY...,7.0,43.5,716,"[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
22951,MRGQVGDLSPQQQEALARFRETLQDLLPTLPKADDYFLLRWLRARN...,7.0,48.0,402,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
22952,MALKRLTRQRKAILEVVRQARHHPDAAWIYQEVRKRVPKVSLGTIY...,7.0,80.0,130,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
22953,MGEQQLDCALDLMRRLPPQHCDKNLTDLIDLCPHLVDDLLSTIDQP...,7.0,46.9,269,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
22954,MSEKLPFKVADISLAEWGRKAIEIAENEMPGLMKMRELYGQSKPLK...,7.0,50.6,432,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


si met la transformation dans le dataframe : le kernel dies
Si met avant, dans le panda, les dimensions sont pas les bonnes (peut être transposer ??)

## Create 1d conv net

In [12]:
train_df.sample(5)

,protein_sequence,pH,tm,length,encoded_sequence
15623,MATVTNFKLVTPPESSRADKPGATKASDAFQEKKSVSVNYDRGEHE...,7.0,59.9,821,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
21468,MFVPRSLKIKRSSNDDLKSGEAKKSKPEAGGLQVEGDRDTPVHTSV...,7.0,49.8,618,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
21105,MANFDLTRINCQFLDRHLTFPLLEFLCGKEIYNQQELLEYILETVN...,7.0,39.5,433,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
11852,FSGRIRKKLRLAGDQRNASYPHSLQFYLQPPTENISLTEFENLAFD...,7.0,48.6,502,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3624,MTAINRILIVDDEDNVRRMLSTAFALQGFETHCANNGRTALHLFAD...,7.0,50.6,460,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


1. get DataLoader from train_dl

In [13]:
class EnzymesDataset(Dataset):
 
  def __init__(self,df):
    
    # the Amino acid sequences as an int array
    sequence= df['encoded_sequence']
    # numerical : pH and length
    numerical = df.iloc[:,[1,3]].values
    print(numerical.shape)
    # y : the target (tm)
    y=df.iloc[:,2].values
    print(sequence.shape)
    print(y.shape)
    #creta tensors from the numpy arrays
    self.x_sequence=torch.tensor(sequence)
    self.y=torch.tensor(y,dtype=torch.float32)
    self.num=torch.tensor(numerical,dtype=torch.float32)
   
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.x_sequence[idx],self.y[idx] , self.num[idx]


In [14]:
train_df.iloc[:,[1,3]].values

array([[  7., 424.],
       [  7.,  87.],
       [  7., 823.],
       ...,
       [  7., 269.],
       [  7., 432.],
       [  7., 546.]])

In [15]:
# hyperparameters
batch_size = 128
learning_rate = 0.001 # Suggested for Adam
num_epochs = 10


a faire : apres avoir fait one hot encoding, trouver comment mettre l'info de plusieurs channels dans le dataframe, sans qu'il mette d'erreur sur la taille. 
Voir comment mettre un tableau = 1 aa puis la longueur de la ligne = longueur totale (juste transposer ?)


class MyLoss(torch.nn.Module):
    def __init__(self, batch_size, classes):
        super(MyLoss, self).__init__()
        # define some attributes
        self.y_true_one_hot = torch.FloatTensor(batch_size, classes, length)

    def forward(self, y_pred, y_true):
        with torch.no_grad():
            self.y_true_one_hot.zero_().scatter_(1, y_true, 1) # permet one hot encoding
        # do some operations
        return loss


Or use cross entropy loss ?

In [16]:
val_df

,protein_sequence,pH,tm,length,encoded_sequence
0,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,265,"[[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
1,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,1451,"[[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,48.4,380,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,AALANSDISVALIEPNTAQQPPLGKVSCNEFDTRVSAITAQSEALL...,7.0,43.3,380,"[[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,AALPQDEKLITGQLDNGLRYMIYPHAHPKDQVNLWLQIHTGSLQEE...,7.0,45.7,904,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
5734,YSNVEKGAAARARCWTSGNGRPAQWWQEGDQVTRGKYFYECRRGQL...,7.0,39.9,349,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,..."
5735,YTIKEDESFLQQPHYASQEQLEDLFAGLEKAYPNQAKVHFLGRSLE...,7.0,42.9,1380,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5736,YVAFKGLLEADLDRSLLFYARALAEGRPSPRGEFAFRLAQGGAETK...,7.0,86.0,378,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5737,YWFPAEEMRTRNNVNNCFKKPAFANLLRFPQLYPFLCRADFIKVAA...,7.0,48.6,300,"[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."


In [17]:
# create pytorch dataframes
train_d = EnzymesDataset(train_df)
val_d = EnzymesDataset(val_df)


# create pytorch dataloaders
train_dl = torch.utils.data.DataLoader(train_d, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_d, batch_size=batch_size, shuffle=True)

(22956, 2)
(22956,)
(22956,)
(5739, 2)
(5739,)
(5739,)


/var/folders/zp/t_7t2xyj4n760sxfr2jkq6z40000gn/T/ipykernel_2998/3031152663.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646703877/work/torch/csrc/utils/tensor_new.cpp:233.)
  self.x_sequence=torch.tensor(sequence)


In [33]:
class Conv1D_OneChannel(nn.Module):

    def __init__(self):
        super().__init__()
        

        self.prot_seq_one_pooling = nn.Sequential(

            #With pooling only at the end (seen in paper)

            nn.Conv1d(20, 64,kernel_size=8, stride=1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 32, 5, stride=1, padding=1), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.AdaptiveAvgPool1d(32), #argument = output size 
            nn.Conv1d(32, 1, 5, stride=1, padding=1), 
            nn.ReLU(), 
            nn.Dropout(),


        )
        self.numerical = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 64),
            nn.ReLU(),
        )
        self.mlp = nn.Sequential(
            nn.Linear(94, 64),#input devrait être 32 + 64 plutôt non si on utilise MaxPoolId(2)? (était marqué 128 en input avant) Comme on fait le pooling
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 1),
        )

    def forward(self, x,y):
        x = self.prot_seq_one_pooling(x.float())
        y = self.numerical(y)
       
        x = torch.cat((x.squeeze(1), y), 1)
        x = self.mlp(x)
        return x

In [34]:
model = Conv1D_OneChannel()

In [35]:

optimizer = Adam(model.parameters(), lr=learning_rate)
# defining the loss function
criterion = nn.MSELoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    optimizer = optimizer.cuda()
    


In [36]:
def train_epoch(model, optimizer, criterion, train_loader, epoch):
    model.train()
    rho = 0 
    for batch_idx, (seq, target,num) in enumerate(train_loader):
        if torch.cuda.is_available():
            seq = seq.cuda()
            target = target.cuda()
            num = num.cuda()
        optimizer.zero_grad()
        output = model(seq,num)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
        # calculate Spearman's rank correlation coefficient
        p, _ = spearmanr(target.cpu().detach().numpy(), output.cpu().detach().numpy())
        rho += p
        
        print(
            f"Train Epoch: {epoch}-{batch_idx:03d} "
            f"batch_loss={loss.item():0.2e} "
            
        )

    rho = rho / len(train_loader)
    return loss.item() , rho

In [37]:
def test_epoch(model, criterion, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for seq, target,num in test_loader:
       
            if torch.cuda.is_available():
                seq = seq.cuda()
                target = target.cuda()
                num = num.cuda()
            output = model(seq,num)
            test_loss += criterion(output.squeeze(), target).item()  # sum up batch loss
           # calculate pearson correlation 
            p, _ =  spearmanr(target.cpu().detach().numpy(), output.cpu().detach().numpy())
            rho += p
            

    test_loss /= len(test_loader)
    rho = rho / len(test_loader)
    print(
        f"Test set: Average loss: {test_loss:0.2e} "
    )

    return test_loss ,rho

In [38]:
# train and test the model (save it after each epoch)
train_loss_history = []
test_loss_history = []
train_rho_history = []
test_rho_history = []
for epoch in range(1, num_epochs + 1):
    train_loss , rho_train = train_epoch(
        model, optimizer, criterion, train_dl, epoch
    )
    train_loss_history.append(train_loss)
    train_rho_history.append(rho_train)

    
    
    test_loss , rho_test = test_epoch(model, criterion, val_df)
    test_loss_history.append(test_loss)
    test_rho_history.append(rho_test)
    
    torch.save(model.state_dict(), f"2-conv1d_OneHotEncoding_model_{epoch}.pth")


torch.Size([128, 20, 8798])


RuntimeError: Given groups=1, weight of size [64, 21, 8], expected input[128, 20, 8798] to have 21 channels, but got 20 channels instead

In [ ]:
#create loss plot

plt.plot(train_loss_history, label='train loss')
plt.plot(test_loss_history, label='test loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title(' train and test MSE Loss')
plt.legend()
plt.savefig('plots/2-conv1d_OneHotEncoding-Loss.png')


In [ ]:
plt.plot(train_rho_history, label='train rho')
plt.plot(test_rho_history, label='test rho')
plt.xlabel('Epoch')
plt.ylabel('rho')
plt.title(' Spearman\'s rank correlation coefficient')
plt.legend()
plt.savefig('plots/2-conv1d_OneHotEncoding-rho.png')